<figure>
  <img src="https://github.com/v-iashin/video_features/raw/master/docs/_assets/i3d.png" width="300" />
</figure>

The `video_features` library allows you to extract features from
raw videos in parallel with multiple GPUs.
It supports several extractors that capture visual appearance,
optical flow, and audio features. See more details in the
[GitHub repository](https://github.com/v-iashin/video_features).

See more feature extraction examples in colaboratory notebooks:
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Zd7r8uKGLGSxlil4PPnXk_4I3KOsjPpO?usp=sharing) – CLIP
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1HUlYcOJf_dArOcAaR9jaQHuM5CAZiNZc?usp=sharing) – S3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LKoytZmNxtC-EuCp7pHDM6sFvK1XdwlW?usp=sharing) – I3D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1csJgkVQ3E2qOyVlcOM-ACHGgPBBKwE2Y?usp=sharing) – R(2+1)D
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/18I95Rn1B3a2ISfD9b-o4o93m3XuHbcIY?usp=sharing) – RAFT
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17VLdf4abQT2eoMjc6ziJ9UaRaOklTlP0?usp=sharing) – ResNet
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1r_8OnmwXKwmH0n4RxBfuICVBgpbJt_Fs?usp=sharing) – VGGish
* [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16QEwNMqiwqlmBhJCJmNeeEP8gitom0I-?usp=sharing) – [timm](https://huggingface.co/timm) models

In [1]:
! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6

Cloning into 'video_features'...
remote: Enumerating objects: 1462, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 1462 (delta 316), reused 280 (delta 245), pack-reused 1055 (from 2)
Receiving objects: 100% (1462/1462), 288.85 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (798/798), done.
Updating files: 100% (100/100), done.
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
%cd video_features

/workspace/OpenTAD/datasets/video_features


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch
import os
import glob
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

'NVIDIA RTX A4500'

In [7]:
# Select the feature type
feature_type = 'i3d'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['./sample/v_GGSY1Qvo990.mp4']
# args.show_pred = True
# args.stack_size = 24
# args.step_size = 24
# args.extraction_fps = 25
args.flow_type = 'raft'
# args.streams = 'flow'

# Load the model
extractor = ExtractI3D(args)

# Extract features
for video_path in args.video_paths:
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]

Extracting for ./sample/v_GGSY1Qvo990.mp4
rgb
(5, 1024)
[[0.08110569 0.21957687 0.05398615 ... 0.08922368 0.22923173 0.99059284]
 [0.04088495 0.24230701 0.06410656 ... 0.02545086 0.29835206 0.77686161]
 [0.12465777 0.25436386 0.14174932 ... 0.16766945 0.18782155 0.68910944]
 [0.14220038 0.27406254 0.175052   ... 0.06276888 0.15171143 0.22976072]
 [0.21122555 0.18315466 0.27664351 ... 0.14347599 0.24268091 0.0735319 ]]
flow
(5, 1024)
[[2.60536429e-02 3.25302556e-02 7.44597092e-02 ... 6.03993936e-03
  2.07155183e-01 1.64320867e-04]
 [4.67634425e-02 3.59357260e-02 3.69044766e-02 ... 9.13438648e-02
  1.56301945e-01 4.35236767e-02]
 [6.99186176e-02 3.26891802e-02 2.61490773e-02 ... 1.47666529e-01
  4.69152890e-02 1.96367837e-05]
 [5.64589687e-02 2.82810647e-02 4.52930443e-02 ... 2.42515560e-02
  8.94547813e-03 1.30930552e-02]
 [3.39127034e-02 4.41964157e-02 2.52714846e-02 ... 1.88279048e-01
  4.21305262e-02 6.29307423e-03]]
fps
()
19.62
timestamps_ms
(5,)
[ 3261.9775739   6523.95514781  978

In [8]:
! pip freeze

addict==2.4.0
AFSD @ file:///workspace/OpenTAD/opentad/models/roi_heads/roi_extractors/boundary_pooling
Align1D @ file:///workspace/OpenTAD/opentad/models/roi_heads/roi_extractors/align1d
aliyun-python-sdk-core==2.16.0
aliyun-python-sdk-kms==2.16.5
annotated-types==0.7.0
anyio==4.0.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==23.1.0
av==14.4.0
Babel==2.13.1
beautifulsoup4==4.12.2
bleach==6.1.0
blinker==1.4
certifi==2022.12.7
cffi==1.16.0
charset-normalizer==2.1.1
chumpy==0.70
click==8.2.0
colorama==0.4.6
comm==0.2.0
contourpy==1.3.2
crcmod==1.7
cryptography==3.4.8
cycler==0.12.1
Cython==3.1.0
dbus-python==1.2.18
debugpy==1.8.0
decorator==5.1.1
decord==0.6.0
defusedxml==0.7.1
distro==1.7.0
docker-pycreds==0.4.0
einops==0.8.1
entrypoints==0.4
exceptiongroup==1.1.3
executing==2.0.1
fastjsonschema==2.18.1
filelock==3.14.0
fonttools==4.58.0
fqdn==1.5.1
fsspec==2023.4.0
fvcore==0.1.5.post20221221
gdown==5.1.0
gitdb==4.0.12
GitPython

In [9]:
!ls

Dockerfile  README.md	   configs  main.py	models	tests
LICENSE     conda_env.yml  docs     mkdocs.yml	sample	utils


In [10]:
# --- Configuration ---
VIDEO_FOLDER = '../videos'  # Folder containing your .mp4 rally videos
OUTPUT_FOLDER = '../features' # Where to save the .npy files
FEATURE_TYPE = 'i3d' # Keep this as 'i3d'

# --- Create Output Directory ---
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- Load and Configure Feature Extractor ---

# Load the base config for 'i3d'
args = OmegaConf.load(build_cfg_path(FEATURE_TYPE))

# --- Crucial Settings for ActionFormer Compatibility ---
# 1. stack_size: Use the default 64, as this is standard for I3D Kinetics models.
#    While you mentioned 16 frames *per vector*, stack_size refers to the
#    I3D *input* window. The output stride is controlled by step_size.
#    Changing stack_size might make the pre-trained model perform poorly.
args.stack_size = 16 # Keep default unless you have specific reasons

# 2. step_size: Set to 4 to get 1 feature vector per 4 frames.
args.step_size = 4 # CRITICAL for ActionFormer stride=4 requirement

# 3. streams: Use default (null) for two-stream (RGB + Flow) -> 2048D output
args.streams = None # Ensures both rgb and flow are extracted

# 4. extraction_fps: Use default (null) to keep original video FPS.
args.extraction_fps = None

# 5. flow_type: Default 'raft' is fine.
args.flow_type = 'raft'

# 6. on_extraction: Set to 'print' (or None if that works) so the script
#    *returns* the dictionary, allowing us to manually concatenate and save.
#    DO NOT use 'save_numpy' here, as it might only save one stream.
args.on_extraction = 'print' # Or try None if print is too verbose

# 7. device: Set your computation device
args.device = device

# 8. output_path: This argument might be used internally by the extractor
#    if 'on_extraction' was set to save, but we'll define our own saving path.
#    Set it anyway in case the extractor uses it for temporary files.
args.output_path = os.path.join(OUTPUT_FOLDER, "temp_extractor_out") # Or args.tmp_path

# --- Instantiate the Extractor ---
print("Initializing I3D Extractor...")
extractor = ExtractI3D(args)
print("Extractor Initialized.")

# --- Find Video Files ---
video_extensions = ["*.mp4", "*.avi", "*.mov", "*.mkv"] # Add other extensions if needed
video_paths = []
for ext in video_extensions:
    video_paths.extend(glob.glob(os.path.join(VIDEO_FOLDER, ext)))

if not video_paths:
    print(f"Error: No video files found in {VIDEO_FOLDER}")
    exit()

print(f"Found {len(video_paths)} videos to process.")

# --- Process Each Video ---
for video_path in video_paths:
    print(f"\nProcessing: {video_path}")
    try:
        # Extract features - this returns a dictionary
        feature_dict = extractor.extract(video_path)

        # Check if extraction was successful and returned expected keys
        if feature_dict and 'rgb' in feature_dict and 'flow' in feature_dict:
            rgb_features = feature_dict['rgb']
            flow_features = feature_dict['flow']

            # --- Verification ---
            print(f"  RGB shape: {rgb_features.shape}")   # Should be (T, 1024)
            print(f"  Flow shape: {flow_features.shape}")  # Should be (T, 1024)
            if rgb_features.shape[0] != flow_features.shape[0]:
                 print(f"  Warning: Mismatch in temporal length between RGB ({rgb_features.shape[0]}) and Flow ({flow_features.shape[0]}) for {video_path}. Skipping concatenation.")
                 continue
            if rgb_features.shape[1] != 1024 or flow_features.shape[1] != 1024:
                print(f"  Warning: Unexpected feature dimension for {video_path}. RGB: {rgb_features.shape[1]}, Flow: {flow_features.shape[1]}. Expected 1024. Skipping.")
                continue

            # --- Concatenate RGB and Flow Features ---
            # Result shape: (T, 2048) - This is what ActionFormer needs
            combined_features = np.concatenate((rgb_features, flow_features), axis=1)
            print(f"  Combined shape: {combined_features.shape}") # Should be (T, 2048)

            # --- Save to .npy file ---
            video_basename = os.path.basename(video_path)
            video_filename_no_ext = os.path.splitext(video_basename)[0]
            output_npy_filename = f"{video_filename_no_ext}.npy"
            output_filepath = os.path.join(OUTPUT_FOLDER, output_npy_filename)

            np.save(output_filepath, combined_features)
            print(f"  Successfully saved features to: {output_filepath}")

        else:
            print(f"  Warning: Feature extraction failed or did not return 'rgb' and 'flow' keys for {video_path}.")

    except Exception as e:
        print(f"  Error processing {video_path}: {e}")

print("\nFeature extraction completed.")

Initializing I3D Extractor...
Extractor Initialized.
Found 28 videos to process.

Processing: ../videos/S2_Peanut_C1.mp4
  RGB shape: (363, 1024)
  Flow shape: (363, 1024)
  Combined shape: (363, 2048)
  Successfully saved features to: ../features/S2_Peanut_C1.npy

Processing: ../videos/S2_Pealate_C1.mp4
  RGB shape: (292, 1024)
  Flow shape: (292, 1024)
  Combined shape: (292, 2048)
  Successfully saved features to: ../features/S2_Pealate_C1.npy

Processing: ../videos/S2_CofHoney_C1.mp4
  RGB shape: (202, 1024)
  Flow shape: (202, 1024)
  Combined shape: (202, 2048)
  Successfully saved features to: ../features/S2_CofHoney_C1.npy

Processing: ../videos/S3_CofHoney_C1.mp4
  RGB shape: (219, 1024)
  Flow shape: (219, 1024)
  Combined shape: (219, 2048)
  Successfully saved features to: ../features/S3_CofHoney_C1.npy

Processing: ../videos/S3_Hotdog_C1.mp4
  RGB shape: (212, 1024)
  Flow shape: (212, 1024)
  Combined shape: (212, 2048)
  Successfully saved features to: ../features/S3_Hot